# SVI Part III: ELBO Gradient Estimators 

**This Tutorial is adapted from [https://pyro.ai/examples/svi_part_iii.html](https://pyro.ai/examples/svi_part_iii.html)*

We've defined a Pyro model with observations ${\bf x}$ and latents ${\bf z}$ of the form $p_{\theta}({\bf x}, {\bf z}) = p_{\theta}({\bf x}\,|\,{\bf z})\cdot p_{\theta}({\bf z})$. We've also defined a Pyro guide (i.e. a variational distribution) of the form $q_{\phi}({\bf z})$. Here $\theta$ and $\phi$ are variational parameters for the model and guide, respectively.

We'd like to maximize the log evidence $\log p_{\theta}({\bf x})$ by maximizing the ELBO (the evidence lower bound) given by

${\rm ELBO} \equiv \mathbb{E}_{q_{\phi}({\bf z})} \left [\log p_{\theta}({\bf x}, {\bf z}) - \log q_{\phi}({\bf z})\right]$

To do this we are going to take (stochastic) gradient steps on the ELBO in the parameter space $\{\theta,\phi\}$ So we need to  compute unbiased estimates of

$\nabla_{\theta,\phi}\, {\rm ELBO} = \nabla_{\theta,\phi}\,\mathbb{E}_{q_{\phi}({\bf z})} \left [\log p_{\theta}({\bf x}, {\bf z}) - \log q_{\phi}({\bf z})\right]$

How do we do this for general stochastic functions **model()** and **guide()**? To simplify notation let's generalize our discussion a bit and ask how we can compute gradients of expectations of an arbitrary cost function $f({\bf z})$. Let's also drop any distinction between $\theta$ and $\phi$. So we want to compute

$\nabla_{\phi} \, \mathbb{E}_{q_{\phi}({\bf z})} \left[\, f_{\phi}({\bf z}) \,\right]$

Let's start with the easiest case.

### Easy Case: Reparameterizable Random Variables

Suppose that we can reparameterize things such that

$\mathbb{E}_{q_{\phi}({\bf z})} \left [\,f_{\phi}({\bf z}) \,\right] = \mathbb{E}_{q({\bf \epsilon})} \left [\,f_{\phi}(g_{\phi}({\bf \epsilon}))\, \right]$

Crucially we have moved all the $\phi$ dependence inside of the exectation; $q(\epsilon)$ is a fixed distribution with no dependence on $\phi$. This kind of reparameterization can be done for many distributions (e.g. the normal distribution). In this case we can pass the gradient straight through the expectation to get

$\nabla_{\phi} \, \mathbb{E}_{q({\bf \epsilon})} \left [\, f_{\phi}(g_{\phi}({\bf \epsilon})) \,\right]= \mathbb{E}_{q({\bf \epsilon})} \left[\,\nabla_{\phi} \, f_{\phi}(g_{\phi}({\bf \epsilon})) \,\right]$

Assuming $f(.)$ and $g(.)$ are sufficiently smooth, we can get unbiased estimates of the gradient of interest by taking a Monte Carlo estimate of this expectation.

### Tricky Case: Non-reparameterizable Random Variables

What if we cannot do the above reparameterization? Unfortunately this is the case for many distributions of interest, including all discrete distributions. In this case our estimator takes a bit more complicated form. We begin by expanding the gradient of interest as

$ \nabla_{\phi}\, \mathbb{E}_{q_{\phi}({\bf z})} \left [\,f_{\phi}({\bf z}) \, \right]= \nabla_{\phi}\, \int d{\bf z} \; q_{\phi}({\bf z}) \cdot f_{\phi}({\bf z})$

and use the chain rule to write this as

$\int d{\bf z} \; \left \{ \nabla_{\phi}\, q_{\phi}({\bf z}) \cdot f_{\phi}({\bf z}) + q_{\phi}({\bf z}) \cdot \nabla_{\phi}\, f_{\phi}({\bf z})\right \}$

At this point we run into a problem. We know how to generate samples from $q(.)$ --- we just run the guide forward --- but $\nabla_{\phi}\,q_{\phi}({\bf z})$ isn't even a valid probability density. So we need to massage this formula so that it's in the form of an expectation w.r.t. $q(.)$. This is easily done using the identity

$\nabla_{\phi}  q_{\phi}({\bf z}) = q_{\phi}({\bf z})\, \nabla_{\phi}\, \log q_{\phi}({\bf z})$,

which allows us to rewrite the gradient of interest as

$\mathbb{E}_{q_{\phi}({\bf z})} \left [\, (\nabla_{\phi} \log q_{\phi}({\bf z})) \cdot f_{\phi}({\bf z}) + \nabla_{\phi}\, f_{\phi}({\bf z}) \,\right]$

This form of the gradient estimator—variously known as the REINFORCE estimator or the score function estimator or the likelihood ratio estimator—is amenable to simple Monte Carlo estimation.